In [1]:
from google.colab import files
from google.colab import drive
drive.mount('/content/gdrive')
root_path = 'gdrive/My Drive/'

import os
os.environ["WANDB_DISABLED"] = "true"

import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
! nvcc --version
!nvidia-smi -L
device

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0
GPU 0: NVIDIA A100-SXM4-40GB (UUID: GPU-940db91b-9f19-e0d3-eb2c-41ec78af8694)


device(type='cuda')

In [2]:
!pip install -U transformers
#!pip install transformers==4.28.0
!pip install datasets
!pip install snscrape
!pip install --upgrade accelerate
!pip install datasets evaluate
!pip install accelerate -U

from scipy.spatial.distance import cosine
from transformers import AutoModel, AutoTokenizer
import numpy as np
import random
import csv
import pandas as pd
from sklearn.model_selection import StratifiedKFold
import tensorflow as tf
import datasets
from datasets import Dataset, DatasetDict
import evaluate
from sklearn.metrics import classification_report, f1_score

import sys, os
import torch
import json
import re

In [3]:
# preprocess functions
import re

def remove_url(text):
    return ' '.join(re.sub("(\w+:\/\/\S+)"," ", text).split())

def remove_username(text):
    return ' '.join(re.sub("([@][A-Za-z0-9_]+)"," ", text).split())

def remove_at_mark(text):
    return re.sub(r'[@]', ' ', text)

def remove_tag_mark(text):
    return re.sub(r'[#]', ' ', text)

def replace_hashtags_with_segments (text):
   regex = "#(\w+)"
   text_split = text.split()
   sent= ''
   strip_word =' '
   for word in text_split:
     hashtag= re.findall(regex, word)
     #print(hashtag)
     #print(strip_word)

     if (len(hashtag)) == 0 :
        sent = sent + ' ' + word
     if (len(hashtag))!=0:
        strip_word = word.replace(('#'+hashtag[0]), ' ')
        #print (hashtag[0])
        #print(strip_word)
        try :
           key = '#'+ hashtag[0]
           segmented = segmented_dict [key]
           if type(segmented) == str:
                    #segmented = re.sub(r'[|]', ' ', segmented)
                    #segmented = re.sub(r'[,]', ' ', segmented)
                    sent= sent + ' ' + segmented
           if type(segmented) == list:
                    for k in segmented:
                      sent = sent +' ' + k
        except:
           sent = sent + ' ' +  hashtag[0]
   return sent + ' ' + strip_word

def control_tag_mark(text):
     s = ' '
     for word in text.split():
            word = re.sub(r'[#]', ' #', word)
            s = s + ' ' + word
     return s

def remove_punctuation_marks(text):
   punc = '''!()-[]{};:'"\,<>./?$%^&*~'''
   for i in text:
       if i in punc:
         text = text.replace(i, " ")
   return text



def remove_underline(text):
  punc = '''_'''
  for i in text:
       if i in punc:
         text = text.replace(i, "")
  return text

def remove_hashtag(text):
    return ' '.join(re.sub("([#][A-Za-z0-9_]+)"," ", text).split())

def remove_turkish_hashtag(text):
    return ' '.join(re.sub("([#][\w+_]+)"," ", text).split())

def find_emoji(text):
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return re.findall(emoji_pattern, text) # no emoji

def remove_emoji(text):
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return (emoji_pattern.sub(r'', text)) # no emoji

def remove_arabic_hashtags(text):
   return ' '.join(re.sub('([#][\u0600-\u06ff_]+)', " ", text).split())

def remove_emoji_duplicate (text):
      emoji =[]
      rest = ''
      main_text = []
      for w in text :
         x = find_emoji(w)
         if len (x) != 0 :
            emoji.append (x[0])
      #print(emoji)
      rest = remove_emoji(text)
      emoji = list(set(emoji))
      for e in emoji :
        try:
          rest = rest + " " + e
        except:
          rest = rest
      main_text.append(rest)
      return main_text[0]

def lower_case(text):
    text = text.replace("I", "ı")
    text = text.replace("İ", "i")
    return text.lower()

In [4]:
def seed_everything(seed):
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(0)

In [5]:
# define data path
# Please add the adress of datasets on your local drive

path_Turkish_data     =  '/content/gdrive/My Drive/Colab Notebooks/my_hate_speech/all_data_work/final_all_data/HS_GitHub_data_codes/data/df_turkish_cats_targets_degrees_final_with_translated_all.csv'
path_arabic_data      =  '/content/gdrive/My Drive/Colab Notebooks/my_hate_speech/all_data_work/final_all_data/HS_GitHub_data_codes/data/df_arabic_cats_targets_degrees_final.csv'
path_synthetic_data   =  '/content/gdrive/My Drive/Colab Notebooks/my_hate_speech/all_data_work/final_all_data/HS_GitHub_data_codes/data/synthetic_data_all_new.xlsx'

path_all_emoji =         '/content/gdrive/My Drive/Colab Notebooks/my_hate_speech/all_data_work/final_all_data/HS_GitHub_data_codes/data/all_emoji_dict.json'
path_hashtag_segment  =  '/content/gdrive/My Drive/Colab Notebooks/my_hate_speech/all_data_work/final_all_data/HS_GitHub_data_codes/data/all_hashtag_segmented_tr.json'

In [6]:
# laod emoji_dict and hashtag_dict

f = open(path_all_emoji)
data = json.load(f)
emoji_list = data['emojis']

f = open(path_hashtag_segment)
segmented_dict = json.load(f)

In [7]:
# determine which preprocess steps must be apply on data

def pre_process_tweets(text_list):
   cleaned_text =[]
   for s in text_list:
       #s = lower_case(s)
       s = s.lower()
       s = remove_username(s)
       s = remove_at_mark(s)
       s = remove_url(s)
       s = remove_tag_mark(s)
       #s = control_tag_mark(s)
       s = remove_punctuation_marks(s)
       s = remove_underline(s)
       #s = replace_hashtags_with_segments(s)
       #s = remove_tag_mark(s)
       cleaned_text.append(s)
   return cleaned_text

In [8]:
import pandas as pd

df_syn = pd.read_excel(path_synthetic_data)

df_syn = df_syn.iloc[:,[4,5,6,7,9]]

print(df_syn['CatComb6class'].value_counts())
print(df_syn['CatComb4class'].value_counts())
print(df_syn['CatComb2class'].value_counts())

df_syn= df_syn.sample(frac = 1)
df_syn


CatComb6class
{0}    2737
{4}     228
{5}     210
{2}      63
{1}      35
{3}      11
Name: count, dtype: int64
CatComb4class
{0}    2737
{3}     439
{2}      72
{1}      36
Name: count, dtype: int64
CatComb2class
{0}    2737
{1}     547
Name: count, dtype: int64


,Text,CatComb6class,CatComb4class,CatComb2class,translated_text_arabic
1293,"İranlılar, zengin bir edebi ve sanatsal geçmiş...",{0},{0},{0},يمتلك الإيرانيون تاريخاً أدبياً وفنياً ثرياً.
2868,"Burmalılar, geleneksel yemeklerde pirinç, sebz...",{0},{0},{0},يفضل البورميون استخدام الأرز والخضروات والأسما...
2103,"Danimarkalılar, özgürlüğü ve eşitliği savunan ...",{0},{0},{0},يتمتع الدنماركيون ببنية اجتماعية تدافع عن الحر...
543,ülkemde siyonist istemiyorum #ülkemdesiyonist...,{2},{2},{1},لا أريد الصهاينة في بلدي
648,İsrail ve komşu ülkeler arasında kültürel işbi...,{0},{0},{0},يتزايد التعاون الثقافي بين إسرائيل والبلدان ال...
...,...,...,...,...,...
835,Bu sezon ligdeki takımlar arasında büyük bir r...,{0},{0},{0},هناك تنافس كبير بين الفرق في الدوري هذا الموسم.
3264,"Emeviler, İslam tarihindeki ilk hanedanlık olu...",{0},{0},{0},الأمويون هم السلالة الأموية الأولى في التاريخ ...
1653,"Azerbaycanlılar, özellikle Bakü ve diğer büyük...",{0},{0},{0},على الرغم من أن الأذربيجانيين يعيشون نمط حياة ...
2607,"Sabataycılık, zamanla kültürel ve dini bir kim...",{0},{0},{0},ومع مرور الوقت، شكلت السبطية هوية ثقافية وديني...


In [9]:
import pandas as pd

df_turkish = pd.read_csv(path_Turkish_data)
df_turkish = df_turkish.iloc[:,[2, 27, 29, 31, 32,38, 39]]

print(df_turkish['CatComb6class'].value_counts())
print(df_turkish['CatComb4class'].value_counts())
print(df_turkish['CatComb2class'].value_counts())

df_turkish

CatComb6class
{0}                   6098
{1}                   1207
{4}                    808
{5}                    692
{2}                    616
                      ... 
{0, 1, 2, 3, 4}          4
{0, 1, 2, 4, 5}          3
{1, 2, 4, 5}             2
{0, 1, 3, 4, 5}          1
{0, 1, 2, 3, 4, 5}       1
Name: count, Length: 62, dtype: int64
CatComb4class
{0}             6046
{3}             1639
{2}             1352
{1}             1078
{2, 3}           252
{0, 2}           244
{0, 1}           214
{1, 2}           208
{0, 2, 3}        194
{0, 3}           186
{0, 1, 2}        121
{0, 1, 2, 3}     106
{1, 3}           103
{1, 2, 3}         63
{0, 1, 3}         54
Name: count, dtype: int64
CatComb2class
{0}       5678
{1}       5257
{0, 1}     925
Name: count, dtype: int64


,Text,CatComb6class,CatComb4class,CatComb2class,CatComb2classWeighted,Div,translated_text_arabic
0,@username Biz önceleri Alevi yoldaşlarımıza di...,{0},{0},{0},{0},test,كنا نقول لرفاقنا العلويين لقد وقعتم في حب جلاد...
1,Bu millet bir insana alevi diye oy vermedi bun...,{0},{0},{0},{0},test,هذه الأمة لم تصوت لشخص لأنه علوي، لا أستطيع أن...
2,Sırf alevi diye eli oya gitmeyen binlerce insa...,{0},{0},{0},{0},test,هناك الآلاف من الناس الذين لا يصوتون لمجرد أنه...
3,@username @username Sonuç normal değil mi Alev...,{0},{0},{0},{0},train,أليست النتيجة طبيعية؟ لقد ظنوا أن الرئيس العلو...
4,@username Sebep ne alevi olmasimi.celladina aş...,{0},{0},{0},{0},train,ولكن ما هو السبب؟ حتى هذه النتائج في رأيي هي ن...
...,...,...,...,...,...,...,...
11855,Yunanistan en gelişmiş F-16 Viper serisi Block...,{0},{0},{0},{0},train,بدأت اليونان في شراء سلسلة طائرات F-16 فايبر ب...
11856,Bak buda kıvırcık saçlı..ya yunan yada ermeni ...,{3},{2},{1},{1},train,انظروا، هذه لديها شعر مجعد إما يوناني أو أرميني
11857,@username Geri istiyormusunuz diye sordukların...,{0},{0},{0},{0},train,عندما سألوا إن كنتم تريدون استعادتها لم يكن إي...
11858,Yunan Sahil Güvenlik yine taciz etti\n\nhttps:...,{0},{0},{0},{0},test,مضايقة خفر السواحل اليوناني مرة أخرى


In [10]:
import pandas as pd

df_arabic = pd.read_csv(path_arabic_data)
df_arabic = df_arabic.iloc[:,[2, 27, 29, 31, 32, 38]]

df_arabic

,Text,CatComb6class,CatComb4class,CatComb2class,CatComb2classWeighted,Div
0,مقال جميل لد. \n@username\n ، هل تعود تركيا لل...,{0},{0},{0},{0},train
1,خلال 20سنة وكل انجازات #اردوغان ومئات المشاريع...,{0},{0},{0},{0},train
2,رئيسة حزب الجيد #ميرال_أكشنار\nأعتقد أن #أردو...,{0},{0},{0},{0},train
3,بحد أدنى 15 ألف ليرة تركية.. #أردوغان يعلن زيا...,{0},{0},{0},{0},test
4,أعلن الرئيس #أردوغان، الثلاثاء، زيادة رواتب مو...,{0},{0},{0},{0},train
...,...,...,...,...,...,...
2664,فؤاد أوقطاي، نائب الرئيس #أردوغان:\n\nلدينا قو...,{0},{0},{0},{0},train
2665,العنوسه عندهم ٦٠٪\nال٤٠٪ متزوجين بنغال وهنود و...,{2},{2},{1},{1},train
2666,من يختار الأتراك بين #أردوغان و #كلجدار_أوغلو؟...,{0},{0},{0},{0},train
2667,موتوا بغيظكم \nلهذا أحببنا #اردوغان \nخير النا...,{0},{0},"{0, 1}",{0},train


In [11]:
# select numumber of classes  [2, 4, 6]
num_class = 2
k_fold = 11

if num_class == 2:
     column_select = 'CatComb2class'
     labels = ['No hate Speech', 'Hate Speech']
     sentiment_mapping =  {'No hate Speech', 'Hate Speech'}
     hub_id = 'HrantDinkFoundation/'
     model_id = 'hs-2class-prediction'



if num_class == 4:
     column_select = 'CatComb4class'

     labels = ['No Hate Detected',
               'Mild Level (Discriminatory Discourse)',
               'Moderate Level (Exaggeration, Generalization, Attribution, Distortion, Symbolization)',
               'Severe Level (Swearing, Insult, Defamation, Dehumanization, Threat of Enmity/War/Attack/Murder/Harm)']

     hub_id = 'HrantDinkFoundation/'
     model_id = 'hs-4class-prediction'

     sentiment_mapping = {'No Hate Detected',
                          'Mild Level (Discriminatory Discourse)',
                          'Moderate Level (Exaggeration, Generalization, Attribution, Distortion, Symbolization)',
                          'Severe Level (Swearing, Insult, Defamation, Dehumanization, Threat of Enmity/War/Attack/Murder/Harm)',
                          }


if num_class == 6:
     column_select = 'CatComb6class'
     labels = ['No Hate Detected',
               'Mild Level (Discriminatory Discourse)',
               'Moderate Level (Exaggeration, Generalization, Attribution, Distortion)',
               'Elevated Level (Symbolization)',
               'Severe Level (Swearing, Insult, Defamation, Dehumanization)',
               'Extreme Level (Threat of Enmity/War/Attack/Murder/Harm)']

     hub_id = 'HrantDinkFoundation/'
     model_id = 'hs-6class-prediction'

     sentiment_mapping = {'No Hate Detected',
                          'Mild Level (Discriminatory Discourse)',
                          'Moderate Level (Exaggeration, Generalization, Attribution, Distortion)',
                          'Elevated Level (Symbolization)',
                          'Severe Level (Swearing, Insult, Defamation, Dehumanization)',
                          'Extreme Level (Threat of Enmity/War/Attack/Murder/Harm)',
                          }




In [12]:
# select which data used in training

# 'only-tr' : use only Turkish data
# 'only-ar' : use only arabic data
# 'tr-syn'  : use Turkish and synthetic data
# 'ar-syn'  : use arabic and synthetic data
# 'ar-translated-syn'  : use arabic, synthetic and the translated column of Turkish data

mode_train = ['only-tr', 'only-ar', 'tr-syn' , 'ar-syn', 'ar-translated-syn']
mode = mode_train [2]

In [13]:
if mode =='only-tr':
    model_lang = 'tr'
    model_name = 'dbmdz/bert-base-turkish-uncased'
    hub_id = hub_id + 'turkish-' + model_id

    df_train = df_turkish[(df_turkish.Div=='train')]
    df_test  = df_turkish[(df_turkish.Div=='test')]

if mode =='only-ar' :
     model_lang = 'ar'
     model_name = 'aubmindlab/bert-base-arabert'

     hub_id = hub_id + 'arabic-' + model_id
     df_train = df_arabic[(df_arabic.Div=='train')]
     df_test  = df_arabic[(df_arabic.Div=='test')]


if mode == 'tr-syn':
    model_lang = 'tr'
    model_name = 'dbmdz/bert-base-turkish-uncased'
    hub_id = hub_id + 'turkish-' + model_id
    df_train = df_turkish[(df_turkish.Div=='train')]
    df_test  = df_turkish[(df_turkish.Div=='test')]

    df_train = pd.concat([df_train, df_syn], ignore_index=True)


if mode == 'ar-syn':
    model_lang = 'ar'
    model_name = 'aubmindlab/bert-base-arabert'
    hub_id = hub_id + 'arabic-' + model_id

    df_train = df_arabic[(df_arabic.Div=='train')]
    df_test  = df_arabic[(df_arabic.Div=='test')]

    df_syn = df_syn.drop (columns=['Text'])
    df_syn.rename(columns={'translated_text_arabic': 'Text'}, inplace=True)
    df_train = pd.concat([df_train, df_syn], ignore_index=True)


if mode == 'ar-translated-syn':
    model_lang = 'ar'
    model_name = 'aubmindlab/bert-base-arabert'
    hub_id = hub_id + 'arabic-' + model_id

    df_turkish = df_turkish.drop (columns=['Text'])
    df_turkish.rename(columns={'translated_text_arabic': 'Text'}, inplace=True)

    df_syn = df_syn.drop (columns=['Text'])
    df_syn.rename(columns={'translated_text_arabic': 'Text'}, inplace=True)

    df_train_arabic = df_arabic[(df_arabic.Div=='train')]
    df_test  = df_arabic[(df_arabic.Div=='test')]

    df_train = pd.concat([df_train_arabic , df_turkish, df_syn], ignore_index=True)


In [14]:
df_train


,Text,CatComb6class,CatComb4class,CatComb2class,CatComb2classWeighted,Div,translated_text_arabic
0,@username @username Sonuç normal değil mi Alev...,{0},{0},{0},{0},train,أليست النتيجة طبيعية؟ لقد ظنوا أن الرئيس العلو...
1,@username Sebep ne alevi olmasimi.celladina aş...,{0},{0},{0},{0},train,ولكن ما هو السبب؟ حتى هذه النتائج في رأيي هي ن...
2,"@username @username Müslüman gibi görünmek, si...",{5},{3},{1},{1},train,أَنْ تَظْهَرَ بِمَنْزِلَةِ المُسْلِمِ لِتُقْنِ...
3,Masanın en fedakar en güvenilir partisi @usern...,{0},{0},{0},{0},train,الطرف الأكثر إيثارًا وموثوقية على الطاولة أقول...
4,@username @username Madem düşmanlık yapıyorlar...,{5},{3},{1},{1},train,إذا كانوا معادين، فدعهم يزحفون بل والأسوأ من ذ...
...,...,...,...,...,...,...,...
12767,Bu sezon ligdeki takımlar arasında büyük bir r...,{0},{0},{0},NaN,NaN,هناك تنافس كبير بين الفرق في الدوري هذا الموسم.
12768,"Emeviler, İslam tarihindeki ilk hanedanlık olu...",{0},{0},{0},NaN,NaN,الأمويون هم السلالة الأموية الأولى في التاريخ ...
12769,"Azerbaycanlılar, özellikle Bakü ve diğer büyük...",{0},{0},{0},NaN,NaN,على الرغم من أن الأذربيجانيين يعيشون نمط حياة ...
12770,"Sabataycılık, zamanla kültürel ve dini bir kim...",{0},{0},{0},NaN,NaN,ومع مرور الوقت، شكلت السبطية هوية ثقافية وديني...


In [15]:
df_test

,Text,CatComb6class,CatComb4class,CatComb2class,CatComb2classWeighted,Div,translated_text_arabic
0,@username Biz önceleri Alevi yoldaşlarımıza di...,{0},{0},{0},{0},test,كنا نقول لرفاقنا العلويين لقد وقعتم في حب جلاد...
1,Bu millet bir insana alevi diye oy vermedi bun...,{0},{0},{0},{0},test,هذه الأمة لم تصوت لشخص لأنه علوي، لا أستطيع أن...
2,Sırf alevi diye eli oya gitmeyen binlerce insa...,{0},{0},{0},{0},test,هناك الآلاف من الناس الذين لا يصوتون لمجرد أنه...
5,@username Yok Alevi Türkmenler yok Laz kardesl...,{0},{0},{0},{0},test,لا تركمان علويون، لا إخوان لاز، لا إخوان أكراد...
8,Kürtleri şımartmak dediği de terörist olarak ...,{0},{0},{0},{0},test,يتم تدليل الأكراد بعدم اتهامهم بالإرهاب، ولا ش...
...,...,...,...,...,...,...,...
11847,@username İhanet içinde degilseler madem öyle ...,{0},{0},{0},{0},test,إذا لم يكونوا في خيانة، فلماذا احتلوا الأراضي ...
11852,"@username Yunanistan kaşınıyor, biz neyi bekli...",{5},{3},{1},{1},test,اليونان تحك، ماذا ننتظر؟ ماذا ننتظر منهم أن يف...
11853,@username @username #YunanTohumları\n@username 🇹🇷,{3},{2},{1},{1},test,#البذور اليونانية 🇹🇷
11854,@username @username diyor ki;helalleşelim…\nAl...,{3},{2},{1},{1},test,يقول: تعالوا نصنع السلام ... في سبيل الله تعال...


In [16]:
# Selecet the samples of training and testing data where the majority voting is clear

if num_class == 2:
     column_select = 'CatComb2class'
     df_train = df_train[df_train.CatComb2class != set()].copy()
     df_test  = df_test[df_test.CatComb2class != set()].copy()

     df_train = df_train[df_train.CatComb2class.apply(len)==3]
     df_test  = df_test[df_test.CatComb2class.apply(len)==3]


if num_class == 4:
     column_select = 'CatComb4class'
     df_train = df_train[df_train.CatComb4class != set()].copy()
     df_test  = df_test[df_test.CatComb4class != set()].copy()

     df_train = df_train[df_train.CatComb4class.apply(len)==3]
     df_test  = df_test[df_test.CatComb4class.apply(len)==3]

if num_class == 6:
     column_select = 'CatComb6class'
     df_train = df_train[df_train.CatComb6class != set()].copy()
     df_test  = df_test[df_test.CatComb6class != set()].copy()

     df_train = df_train[df_train.CatComb6class.apply(len)==3]
     df_test  = df_test[df_test.CatComb6class.apply(len)==3]

In [17]:
# preprocess train data

import ast

text = df_train['Text'].tolist()
label = df_train[column_select].tolist()
label = [int(i.strip('{}')) for i in label]

text = pre_process_tweets(text)
train_data = pd.DataFrame(list(zip(text, label)), columns =['text', 'label'])
train_data= train_data.astype({'label':'int'})


print(train_data['label'].value_counts())
train_data


label
0    7293
1    4737
Name: count, dtype: int64


,text,label
0,sonuç normal değil mi alevi başkan kürt ortak ...,0
1,sebep ne alevi olmasimi celladina aşık bir top...,0
2,müslüman gibi görünmek sizi müslüman olduğuna...,1
3,masanın en fedakar en güvenilir partisi si bun...,0
4,madem düşmanlık yapıyorlar sürünsünler hatta d...,1
...,...,...
12025,bu sezon ligdeki takımlar arasında büyük bir r...,0
12026,emeviler i̇slam tarihindeki ilk hanedanlık ol...,0
12027,azerbaycanlılar özellikle bakü ve diğer büyük...,0
12028,sabataycılık zamanla kültürel ve dini bir kim...,0


In [18]:
# preprocess test data

text = df_test['Text'].tolist()
label = df_test[column_select].tolist()
label = [int(i.strip('{}')) for i in label]

text = pre_process_tweets(text)
test_data = pd.DataFrame(list(zip(text, label)), columns =['text', 'label'])
test_data= test_data.astype({'label':'int'})



print(test_data['label'].value_counts())
test_data


label
0    1122
1    1067
Name: count, dtype: int64


,text,label
0,biz önceleri alevi yoldaşlarımıza diyorduk siz...,0
1,bu millet bir insana alevi diye oy vermedi bun...,0
2,sırf alevi diye eli oya gitmeyen binlerce insa...,0
3,yok alevi türkmenler yok laz kardeslerim yok k...,0
4,kürtleri şımartmak dediği de terörist olarak s...,0
...,...,...
2184,i̇hanet içinde degilseler madem öyle peki nede...,0
2185,yunanistan kaşınıyor biz neyi bekliyoruz dah...,1
2186,yunantohumları 🇹🇷,1
2187,diyor ki helalleşelim… allah i̇çin bir gelinde...,1


In [19]:
from transformers import AutoTokenizer, DataCollatorWithPadding
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from transformers import ElectraForSequenceClassification,  ElectraTokenizerFast
from transformers import RobertaConfig, RobertaModel, RobertaTokenizer, RobertaTokenizerFast, RobertaForSequenceClassification
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification, Trainer, TrainingArguments, GPT2Config

id2label = {idx:label for idx, label in enumerate(labels)}
label2id = {label:idx for idx, label in enumerate(labels)}

pretrained_model_name = model_name
print(model_name)
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name, do_lower_case=True, force_download=True)
model = AutoModelForSequenceClassification.from_pretrained(pretrained_model_name, num_labels = num_class, force_download=True, id2label=id2label, label2id=label2id ).to(device)   # id2label=id2label, label2id=label2id
model.resize_token_embeddings(len(tokenizer))
for item in emoji_list :
        tokenizer.add_tokens(item)
model.resize_token_embeddings(len(tokenizer))
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


dbmdz/bert-base-turkish-uncased


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/59.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/59.0 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/445M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dbmdz/bert-base-turkish-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


In [20]:
def preprocess_function(examples):
    preprocessed = examples["text"]
    return tokenizer(preprocessed, truncation=True)

In [21]:
def compute_metrics(eval_pred):

    metric4 = evaluate.load("accuracy")
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = metric4.compute(predictions=predictions, references=labels)["accuracy"]
    macro_f1 = (f1_score(labels, predictions, average ="macro"))

    return {"accuracy":accuracy, "macro_f1":macro_f1}



In [22]:
skf = StratifiedKFold(n_splits= k_fold, random_state = 31, shuffle=True)

In [23]:
class CustomTrainer(Trainer):
    def save_model(self, output_dir=None, _internal_call=False):
        if output_dir is None:
            output_dir = self.args.output_dir

        self.model = self.model.to('cuda')

        for param in self.model.parameters():
            if not param.is_contiguous():
                param.data = param.data.contiguous()

        super().save_model(output_dir, _internal_call)

In [24]:
# Let's start training model

for i, (train_index, test_index) in enumerate(skf.split(train_data.text , train_data.label)):

    print("number of samples in train folds: ", len(train_index))
    print("number of samples in test fold  : ", len(test_index))

    test = Dataset.from_pandas(train_data.iloc[test_index].set_index("text"))
    train = Dataset.from_pandas(train_data.iloc[train_index].set_index("text"))
    dataset_dict = DatasetDict({"test": test, "train": train})
    preprocessed_dataset_dict = dataset_dict.map(preprocess_function, batched=True)

    training_args = TrainingArguments(
        "My Hate Speech Detection",
        overwrite_output_dir=True,
        learning_rate=5e-6,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=20,
        num_train_epochs=10,
        weight_decay=0.01,
        eval_strategy="steps",
        metric_for_best_model = 'accuracy',
        logging_steps = 100,
        load_best_model_at_end=True,
        greater_is_better=True,
        #hub_strategy="end",        # These parameter used when you want saved mmodels directly to HuggingFace,
        #push_to_hub=True,          # If you want to save model on huggingFace you must select "True"
        #hub_model_id= hub_id,      # Write your model address on HuggingFace
        #hub_private_repo= False,   # Determine whether the model on Hugging Face should be private or public.
        #hub_token= "......",       # To save model into HuggingFace you need "hub_token" which can take from your own HuggingFace account
    )

    trainer = CustomTrainer(
        model = model,
        args = training_args,
        train_dataset=preprocessed_dataset_dict["train"],
        eval_dataset=preprocessed_dataset_dict["test"],
        tokenizer = tokenizer,
        data_collator = data_collator,
        compute_metrics = compute_metrics,

    )
    trainer.train()
    break

#trainer.save_model()              # If you want save your trained model on HuggingFace


number of samples in train folds:  10936
number of samples in test fold  :  1094


Map:   0%|          | 0/1094 [00:00<?, ? examples/s]

Map:   0%|          | 0/10936 [00:00<?, ? examples/s]

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
<ipython-input-24-c3e5c64fb95e>:31: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `CustomTrainer.__init__`. Use `processing_class` instead.
  trainer = CustomTrainer(


Step,Training Loss,Validation Loss,Accuracy,Macro F1
100,0.585000,0.452335,0.779707,0.775195
200,0.435800,0.388255,0.809872,0.806520
300,0.406000,0.349892,0.839122,0.829492
400,0.374500,0.335967,0.856490,0.848649
500,0.356800,0.337550,0.831810,0.828410
600,0.331200,0.308950,0.863803,0.857561
700,0.305700,0.308242,0.869287,0.861398
800,0.315600,0.296061,0.869287,0.862503
900,0.278300,0.287821,0.880256,0.874041
1000,0.263200,0.316087,0.862888,0.859246


In [25]:
# Let's start testing model

test_ds = Dataset.from_pandas(test_data).map(preprocess_function, batched=True)
test_ds

Map:   0%|          | 0/2189 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 2189
})

In [26]:
predictions = trainer.predict(test_ds).predictions
predictions

array([[ 2.8753042 , -3.0390568 ],
       [ 3.162391  , -3.0890837 ],
       [-0.54966533,  0.28961086],
       ...,
       [-3.4157717 ,  3.1289766 ],
       [-3.5303247 ,  3.4116566 ],
       [ 1.7231959 , -2.33312   ]], dtype=float32)

In [27]:
from sklearn.metrics import confusion_matrix

y_pred = tf.argmax(predictions, axis=1).numpy()
y_test= np.array(test_data['label'])

#print(title)
print(confusion_matrix(y_test, y_pred))
print('-------------------------------------------------------')
print('macro-f1:', f1_score(y_test, y_pred, average ="macro"))
print('accuracy:', f1_score(y_test, y_pred, average ="micro"))

[[961 161]
 [200 867]]
-------------------------------------------------------
macro-f1: 0.8347798452189421
accuracy: 0.8350845134764733


In [28]:
from sklearn.metrics import classification_report, f1_score
from sklearn.metrics import confusion_matrix

con_mat = tf.math.confusion_matrix(labels=y_test, predictions=y_pred, num_classes=num_class)
#print(title)
print(con_mat)

print('-------------------------------------------------------')
print(classification_report(y_test, y_pred, digits=num_class, zero_division=0))

tf.Tensor(
[[961 161]
 [200 867]], shape=(2, 2), dtype=int32)
-------------------------------------------------------
              precision    recall  f1-score   support

           0       0.83      0.86      0.84      1122
           1       0.84      0.81      0.83      1067

    accuracy                           0.84      2189
   macro avg       0.84      0.83      0.83      2189
weighted avg       0.84      0.84      0.83      2189



In [29]:
# save trained model on local drive

if model_lang == 'tr':
          path_save_model = '/content/gdrive/My Drive/Colab Notebooks/my_hate_speech/all_data_work/final_all_data/HS_GitHub_data_codes/trained_models/hs-tr-' + str(num_class) + 'class/'

if model_lang == 'ar':
          path_save_model = '/content/gdrive/My Drive/Colab Notebooks/my_hate_speech/all_data_work/final_all_data/HS_GitHub_data_codes/trained_models/hs-ar-' + str(num_class) + 'class/'


trainer.save_model(path_save_model)

In [30]:
# load trained model from local drive
#Please add the address where the models are saved.

from transformers import pipeline

path_trained_model = path_save_model

def classify_tweet(text):
        prediction = []
        model_pipline = pipeline(task = "text-classification", model = path_trained_model, device="cuda")
        prediction = (model_pipline (text))
        prediction = prediction[0]["label"]
        return prediction

In [31]:
AR_TWEETS_EXAMPLES = [
    "أريد أن يموت اللاجئون",
    "فليمت جميع اللاجئين",
    "كل اللاجئين لصوص",
    "أكره اللاجئين",
    "ارحلوا من بلدنا",
    "هنا مرة أخرى، سلوك اللاجئين المتحرش والمغتصب للاجئين، والآن يجب إزالة القطيع المنحرف من بلادنا",
    "النساء، اجلسن في البيت وإلا ستتعرضن للتحرش، لا تقولوا إنني لم أحذركن",
    "مكان المرأة هو المطبخ",
    "اليونان يجب أن تعرف حدودها، وإلا قد نأتي فجأة في ليلة",
    "لا ينبغي للسوريين أن يصوتوا",
    "ولا يجوز لغير المسلمين أن يكونوا أعضاء في البرلمان",
    "لا ينبغي للاجئين التصويت",
    "لا ينبغي للاجئين الاستفادة من مباحث أمن الدولة",
    "التعليم باللغة الكردية غير مقبول",
    "للاجئين أيضاً الحق في العيش",
    "يتم اتخاذ تدابير شاملة ضمن الإطار القانوني لمنع أنشطة المنظمات الإرهابية",
    "تم الإعلان عن مواعيد الفعاليات الثقافية لهذا العام في إسرائيل",
    "تقوم رائدات الأعمال بافتتاح صالونات تجميل تلبي احتياجات الزبائن من الجاليات المسلمة وغيرها من الجاليات الأخرى",
    "تجري الاستعدادات في الأسواق في سوريا قبل الأعياد الإسلامية والمسيحية",
    "يجب على النساء دعم بعضهن البعض",
    "اشتريت تفاحًا من السوق",
    "طعام السوريين لذيذ جداً",
    "الأكل السوري لذيذ",
    "أحب السوريين كثيراً، فهم شعب متعاطف جداً"
]

TR_TWEETS_EXAMPLES = [
    "Mülteciler ölsün istiyorum.",
    "Bütün mülteciler hırsızdır.",
    "Mültecilerden nefret ediyorum.",
    "Defolun gidin bizim ülkemizden.",
    "Suriyeli sürüleri ülkemize girmeye devam ediyor, bu işgalcilerin artık durması gerekiyor.",
    "Kadınlar dizinizi kırıp evde oturun yoksa tacize uğrarsınız, demedi demeyein!",
    "Kadının yeri mutfaktır.",
    "Yunan haddını bilecek yoksa bir gece ansızın gelebiliriz!",
    "Suriyeliler oy kullanmasın.",
    "Gayrimüslimler milletvekili olmasın.",
    "Mülteciler oy kullanmasın.",
    "Mülteciler sgk'dan yararlanmasın.",
    "Kürtçe eğitim kabul edilemez.",
    "Mültecilerin de yaşamaya hakkı var.",
    "Terör örgütlerinin faaliyetlerini önlemek için hukuki çerçevede kapsamlı önlemler alınmaktadır.",
    "İsrail'de bu yıl düzenlenecek kültürel etkinliklerin tarihleri açıklandı",
    "Kadın girişimciler, hem Müslüman hem de diğer topluluklardan müşterilere hitap eden güzellik salonları açıyor.",
    "Suriye’deki pazarlarda Müslüman ve Hristiyan bayramları öncesinde hazırlıklar yapılıyor.",
    "Bazı köpeklerin kuduz ya da saldırgan olması, tüm köpeklerin öldürülmesi için haklı bir sebep midir? Yasayı geri çek.",
    "Kadınlar birbirlerine destek olmalı.",
    "Pazardan elma aldim.",
    "Suriyelilerin yemekleri çok güzel.",
    "Suriye yemekleri çok lezzetli.",
    "Süriyelileri çok seviyorum, çok sempatik insanlar."
]

if model_lang == 'ar':
      TWEETS_EXAMPLES = AR_TWEETS_EXAMPLES

if model_lang == 'tr':
      TWEETS_EXAMPLES = TR_TWEETS_EXAMPLES

for tweet in TWEETS_EXAMPLES:
    preprocessed_tweet = remove_punctuation_marks(tweet.lower())
    print(preprocessed_tweet)
    sentiment = classify_tweet(preprocessed_tweet)
    print(f"Tweet:    {tweet}")
    print(f"Category: {sentiment}")
    print("-" * 60)


mülteciler ölsün istiyorum 


Device set to use cuda


Tweet:    Mülteciler ölsün istiyorum.
Category: Hate Speech
------------------------------------------------------------
bütün mülteciler hırsızdır 


Device set to use cuda


Tweet:    Bütün mülteciler hırsızdır.
Category: Hate Speech
------------------------------------------------------------
mültecilerden nefret ediyorum 


...

Tweet:    Suriyelilerin yemekleri çok güzel.
Category: No hate Speech
------------------------------------------------------------
suriye yemekleri çok lezzetli 


Device set to use cuda


Tweet:    Suriye yemekleri çok lezzetli.
Category: No hate Speech
------------------------------------------------------------
süriyelileri çok seviyorum  çok sempatik insanlar 


Device set to use cuda


Tweet:    Süriyelileri çok seviyorum, çok sempatik insanlar.
Category: No hate Speech
------------------------------------------------------------
